In [13]:
import openai


with open("../base.apikey", "r") as api_key_file:
    api_key = api_key_file.readline().strip()
    if api_key == "":
        print("No APIKEY!")
        exit()
    openai.api_key = api_key

def talk_gpt(model_name, system_prompt, user_prompt, tempature):
    messages = [
        {
            "role":"system",
            "content":system_prompt
        },
        {
            "role":"user",
            "content":user_prompt
        }
        
    ]
    completion = openai.ChatCompletion.create(
        model=model_name,
        messages=messages,
        n = 1,
        temperature = tempature,
        top_p = 0.95,
        stream = False
    )
    return completion.choices[0]["message"]["content"]


In [24]:
robot_system = '''
You are a robot command interpreter, responsible for translating the natural language to robot command script.

Here are the robot command script format:

# 1. Every command has two parts: command name and arguments. Use "::" between the command name and arguments. Use "," to split each argument.

# 2. The script should starts with start command "START::" and ends with end command "END::"

# 3. One command occupy one line, comment should also be in the dependent line.

# 4. Comment line should start with "//", just like single line comment in C and C++.

Here are the legal commands list, every command are quoted in [] and with explanation behind:

# [ START:: ] the start command, takes no argument, means the script starts here.

# [ END:: ] the end command, takes no argument, means the script ends here.

# [ GO::<direction>,<distance> ] the moving command, ask the robot to move along a direction for given distance. The direction should be one of [Forward, Right, Left, Back], the distance unit is meter. When using this command, the robot will translate (means shift) to the distination, the orientation of the head does not change.

# [ TAG:: ] put a tag in current piosition, takes no argument.

# [ TURN::<direction> ] the turning command, ask the robot's head turns to a new direction. The direction should be one of [Right, Left, Back], Considering the current facing direction as forward, to turn back means to turn to the opposite direction of the current one, to turn left means to turn to the left of the current direction, and to turn right means to turn to the right of the current direction.

You need to obey the rules:

# 1. Only output the robot commands list and legal comment, make sure your output can run as a script directly.

# 2. If there is any natural language content such as comment, explanation content, should be in the comment part.

# 3. If you think you cannot translating the command successfully, output an empty command and explan it in comment.

# 4. If the user didn't mention about "turn to some direction and go forward" or "shift to some direction", use turning and going forward as default instead of shifting.

Here are some input and output examples:

```

[User]:

Please go forward for 2 meters and turn back for 1.5 meters

[Output]:

START::

GO::Forward,2

//Turn back

TURN::Back

GO::Forward,1.5

END::

```

```

[User]:

Please shift right for 2 meters and shift back for 3 meters.

[Output]:

START::

//Go right for 2 meters

GO::Right,2

GO::Back,3

END::

```
'''

In [20]:
res = talk_gpt("gpt-4-0314", robot_system, "Go forward for 1 meter, go back for 2 meters.", 0.2)
print(res)

START::

GO::Forward,1

GO::Back,2

END::


In [21]:
res = talk_gpt("gpt-4-0314", robot_system, "Go forward for 2 meters, turn left and go forward for 0.7 meters, and shift right for 2 meters.", 0.2)
print(res)

START::

GO::Forward,2

TURN::Left

GO::Forward,0.7

GO::Right,2

END::


In [22]:
res = talk_gpt("gpt-4-0314", robot_system, "Walk a square with 3-meter side length.", 0.2)
print(res)

START::

// Walk the first side of the square
GO::Forward,3

// Turn right to walk the second side of the square
TURN::Right
GO::Forward,3

// Turn right to walk the third side of the square
TURN::Right
GO::Forward,3

// Turn right to walk the fourth side of the square
TURN::Right
GO::Forward,3

END::


In [23]:
res = talk_gpt("gpt-4-0314", robot_system, "Turn left and walk 5 meters, then turn back and walk 2 meters, and repeat for 3 times.", 0.2)
print(res)

START::

// Repeat 3 times
// First iteration
TURN::Left
GO::Forward,5
TURN::Back
GO::Forward,2

// Second iteration
TURN::Left
GO::Forward,5
TURN::Back
GO::Forward,2

// Third iteration
TURN::Left
GO::Forward,5
TURN::Back
GO::Forward,2

END::


In [25]:
res = talk_gpt("gpt-4-0314", robot_system, "Try walk a circle-like path, with radius of 3 meters.", 0.2)
print(res)

// A circle-like path cannot be accurately represented with the available commands.
// However, we can approximate it by moving in small increments and turning slightly.
// This script will approximate a circle with a radius of 3 meters.

START::

// Approximate circle with 36 segments
// Each segment length: 2 * pi * 3 / 36 = 0.5236 meters
// Each turn angle: 360 / 36 = 10 degrees

// Repeat 36 times
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Forward,0.5236
TURN::Right
GO::Fo

In [26]:
json_system = '''
You are a JSON formatter AI assistant that helps people to answer questions with JSON format.

You should answer every quesion using JSON.

Here are the rules:
# 1. Your whole answer should be a JSON object.
# 2. Your answer should at least contains 2 parts: Answer and Explanation.
# 3. For the Answer part, you should put a sub json object to describe the answer.
# 4. For the Explanation part, you should try explaning the answer in natural language in 1~3 sentence brefily.

Some examples:

```
User: Tom has 3 apples and 2 oranges, Jimmy gives him 3 more apples and 2 penciles, how many of each kind fruits does Tom has now?

Assiatant: { "Answer": { "Apple":6, "Orange":2 }, "Explanation":"A pencile is not fruit, so pencile is not calculated in." }

```

```
User: Jack is an engineer, David is a soilder, Lily used to be a marketing manager but now she's a lawyer. what's the jobs now of each person above?
Assiatant: { "Answer": [ { "name":"Jack", "job":"engineer" }, { "name":"David", "job":"soilder" }, { "name":"Lily", "job":"lawyer" } ], "Explanation":"Lily isn't a marketing managet anymore." }

```
'''

In [27]:
res = talk_gpt("gpt-4-0314", json_system, "Tom is 18 years old, Shawn is 5 years older than Tom, Tim is 3 years younger than Shawn. How old is each person above?", 0.2)
print(res)

{
  "Answer": [
    { "name": "Tom", "age": 18 },
    { "name": "Shawn", "age": 23 },
    { "name": "Tim", "age": 20 }
  ],
  "Explanation": "Shawn is 5 years older than Tom (18+5=23), and Tim is 3 years younger than Shawn (23-3=20)."
}


In [28]:
res = talk_gpt("gpt-4-0314", json_system, "I have a Macbook pro, which values 3000 dollars. I use it for video editing and programming. I also have a iPad, I can do simple video editing work by iPad but cannot do programming work on it. What can I do with the two devices above?", 0.2)
print(res)

{
  "Answer": {
    "Macbook Pro": ["Video Editing", "Programming"],
    "iPad": ["Simple Video Editing"]
  },
  "Explanation": "You can do video editing and programming on your Macbook Pro, while you can only do simple video editing on your iPad."
}


In [29]:
res = talk_gpt("gpt-4-0314", json_system, "Tony has a box, in the box there is a pen, a can of cola, an envelope, inside the envelope there is a card-pocket, inside the card-pocket there is a bank card, a credit card, a passport. There is two photos sandwiched in the passport. Describe what Tony have and the containing relationship. ", 0.2)
print(res)

{
  "Answer": {
    "Box": {
      "Pen": {},
      "Cola": {},
      "Envelope": {
        "Card-pocket": {
          "Bank card": {},
          "Credit card": {},
          "Passport": {
            "Photos": 2
          }
        }
      }
    }
  },
  "Explanation": "Tony has a box containing a pen, cola, and an envelope. Inside the envelope, there's a card-pocket with a bank card, credit card, and a passport. The passport has two photos sandwiched in it."
}


In [1]:
import os
os.getlogin()

'hongxiao95'

In [2]:
import torch
torch.cuda.is_available()


False

In [7]:
torch.__version__

'1.12.1'

In [9]:
import torch
torch.cuda.is_available()

False